## Pemodelan prediksi Cool and Warm by Season menggunakan RNN

In [1]:
# Mengimport Pustaka yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import itertools
import warnings
warnings.filterwarnings('ignore')

### Data Loading

In [2]:
df=pd.read_excel('Data c&w.xlsx')

In [3]:
df.tail()

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
70,2020,0.5,0.5,0.4,0.2,-0.1,-0.3,-0.4,-0.6,-0.9,-1.2,-1.3,-1.2
71,2021,-1.0,-0.9,-0.8,-0.7,-0.5,-0.4,-0.4,-0.5,-0.7,-0.8,-1.0,-1.0
72,2022,-1.0,-0.9,-1.0,-1.1,-1.0,-0.9,-0.8,-0.9,-1.0,-1.0,-0.9,-0.8
73,2023,-0.7,-0.4,-0.1,0.2,0.5,0.8,1.1,1.3,1.6,1.8,1.9,2.0
74,2024,1.8,1.5,1.1,0.7,0.4,0.2,0.0,-0.1,NaN,NaN,NaN,NaN


### Data Cleaning

In [4]:
df.dropna(inplace=True)

In [5]:
df.tail()

,Year,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
69,2019,0.7,0.7,0.7,0.7,0.5,0.5,0.3,0.1,0.2,0.3,0.5,0.5
70,2020,0.5,0.5,0.4,0.2,-0.1,-0.3,-0.4,-0.6,-0.9,-1.2,-1.3,-1.2
71,2021,-1.0,-0.9,-0.8,-0.7,-0.5,-0.4,-0.4,-0.5,-0.7,-0.8,-1.0,-1.0
72,2022,-1.0,-0.9,-1.0,-1.1,-1.0,-0.9,-0.8,-0.9,-1.0,-1.0,-0.9,-0.8
73,2023,-0.7,-0.4,-0.1,0.2,0.5,0.8,1.1,1.3,1.6,1.8,1.9,2.0


In [6]:
df.set_index('Year', inplace = True)

In [7]:
dates = pd.date_range(start = '1950-01', freq = 'MS', periods = len(df.columns)*12)
dates

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '1961-03-01', '1961-04-01', '1961-05-01', '1961-06-01',
               '1961-07-01', '1961-08-01', '1961-09-01', '1961-10-01',
               '1961-11-01', '1961-12-01'],
              dtype='datetime64[ns]', length=144, freq='MS')

In [9]:
df.head()

,DJF,JFM,FMA,MAM,AMJ,MJJ,JJA,JAS,ASO,SON,OND,NDJ
Year,,,,,,,,,,,,
1950,-1.5,-1.3,-1.2,-1.2,-1.1,-0.9,-0.5,-0.4,-0.4,-0.4,-0.6,-0.8
1951,-0.8,-0.5,-0.2,0.2,0.4,0.6,0.7,0.9,1.0,1.2,1.0,0.8
1952,0.5,0.4,0.3,0.3,0.2,0.0,-0.1,0.0,0.2,0.1,0.0,0.1
1953,0.4,0.6,0.6,0.7,0.8,0.8,0.7,0.7,0.8,0.8,0.8,0.8
1954,0.8,0.5,0.0,-0.4,-0.5,-0.5,-0.6,-0.8,-0.9,-0.8,-0.7,-0.7


In [10]:
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(df)
scaled_data.shape

(74, 12)

In [11]:
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(df) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])  # Target adalah nilai di langkah berikutnya
    return np.array(X), np.array(y)

In [12]:
time_steps = 1
X, y = create_dataset(scaled_data, time_steps)

In [13]:
train_size = int(len(X) * 0.75)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from attention import Attention

np.random.seed(100)

### Pendefinisian Model

In [14]:
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train.shape[1]))

In [15]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [16]:
train_score = model.evaluate(X_train, y_train, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)

print('Train Score: %.4f' % train_score)
print('Test Score: %.4f' % test_score)


Train Score: 0.4136
Test Score: 0.3512


In [17]:
y_pred=model.predict(X_test)

1/1 [==============================] - 0s 217ms/step


In [18]:
y_pred_inv = scaler.inverse_transform(y_pred)
print('Prediksi:', y_pred_inv)

Prediksi: [[-1.8264347  -2.70654    -1.1981453  -0.19689375 -0.75412    -2.7008483
  -1.4831238  -1.6030182  -2.2533722  -1.7567257  -4.6911736  -0.8339442 ]
 [-1.7349746  -3.0131881  -1.2002573  -0.32266882 -0.82144123 -2.8282013
  -0.76755357 -1.590911   -2.3020673  -1.5397929  -4.9640846  -1.3857635 ]
 [-1.9605622  -2.1533608  -1.2228131  -0.64517987 -0.8254246  -2.1953263
  -1.7257786  -1.6385332  -2.2014444  -1.8041013  -3.1573055  -1.1188873 ]
 [-1.7728939  -2.6991074  -1.185471   -0.6228045  -0.92047584 -2.398439
  -0.7773931  -1.3716569  -2.1456282  -1.4898937  -4.059308   -1.9639987 ]
 [-1.7772012  -1.9283042  -1.1188622  -0.9981558  -0.99648976 -1.6321237
  -1.6620936  -1.6757562  -2.0662816  -1.7936511  -2.3577557  -1.3817757 ]
 [-2.0546925  -2.2384825  -1.2132956  -0.56092197 -0.7813915  -2.3124146
  -1.793238   -1.7145455  -2.2789364  -1.8117745  -3.2735004  -0.9960774 ]
 [-1.3979931  -3.6009388  -1.2961994  -0.674069   -0.8118379  -2.972995
  -0.3161469  -1.2030122  -2.28

In [19]:
r2 = r2_score(y_test, y_pred)

print('R² Score: %.2f' % r2)

R² Score: -5.56
